# 데이터 사이언스 100번의 노크(구조화 데이터 처리편) - R



## 참고(Reference) : 「データサイエンティスト協会スキル定義委員」の「データサイエンス100本ノック（構造化データ加工編）」

## 데이터 사이언티스트 소사이어티 깃허브 : https://github.com/The-Japan-DataScientist-Society/100knocks-preprocess

## 데이터 사이언스 100 노크 (구조화된 데이터 처리) URL : https://github.com/The-Japan-DataScientist-Society/100knocks-preprocess/blob/master/docker/work/answer/ans_preprocess_knock_R.ipynb

- 참고: 원래 데이터 사이언티스트 소사이어티(データサイエンティスト協会スキル定義委員)에서 만든 ipynb 파일을 DeepL 에서 일본어에서 영어로 번역한 파일입니다.

- 이 파일을 업데이트한 이유는 초보자부터 고급 엔지니어까지 R을 사용하고자 하는 모든 사람에게 유용한 이 실습을 전파하기 위해서입니다. 

- 이 데이터는 일본어로 작성되었으므로 연습할 때 언어 문제가 발생할 수 있습니다. 그러나 크게 영향을 미치지 않으므로 걱정하지 마십시오.

## 소개
- 먼저 다음 셀을 실행합니다.
- 필요한 라이브러리를 가져오고 데이터베이스에서 데이터를 읽습니다(PostgreSQL).
- 사용할 것으로 예상되는 라이브러리는 다음 셀에서 가져옵니다.
- 사용하려는 다른 라이브러리가 있는 경우 install.packages()를 사용하여 적절하게 설치합니다.
- 이름, 주소 등은 더미 데이터이며 실제 데이터가 아닙니다.

In [ ]:
require("RPostgreSQL")
require("tidyr")
require("dplyr")
require("stringr")
require("caret")
require("lubridate")
require("rsample")
require("recipes")
require("themis")

host <- "db"
port <- Sys.getenv()["PG_PORT"]
dbname <- Sys.getenv()["PG_DATABASE"]
user <- Sys.getenv()["PG_USER"]
password <- Sys.getenv()["PG_PASSWORD"]

con <- dbConnect(PostgreSQL(), host=host, port=port, dbname=dbname, user=user, password=password)
df_customer <- dbGetQuery(con,"SELECT * FROM customer")
df_category <- dbGetQuery(con,"SELECT * FROM category")
df_product <- dbGetQuery(con,"SELECT * FROM product")
df_receipt <- dbGetQuery(con,"SELECT * FROM receipt")
df_store <- dbGetQuery(con,"SELECT * FROM store")
df_geocode <- dbGetQuery(con,"SELECT * FROM geocode")

# 연습 문제

---
> R-001: 영수증 내역 데이터(df_receipt)에서 전체 항목 중 첫 10건을 표시하여 어떤 데이터를 보유하고 있는지 목시으로 확인하라.

---
> R-002: 영수증 내역 데이터(df_receipt)에서 판매일자(sales_ymd), 고객 아이디(customer_id), 상품코드(product_cd), 판매금액(amount)의 순서로 열을 지정하여 10건씩 표시하라.

---
> R-003: 영수증 내역 데이터(df_receipt)에서 매출일자(sales_ymd), 고객 ID(customer_id), 상품코드(product_cd), 매출금액(amount) 순으로 열을 지정하여 10건씩 표시한다. 단, sales_ymdsales_date로 항목명을 변경하면서 추출한다.

---
> R-004: 영수증 내역 데이터(df_receipt)에서 판매일(sales_ymd), 고객 ID(customer_id), 상품 코드(product_cd), 판매 금액(amount) 순으로 열을 지정하고, 다음 조건을 만족하는 데이터를 추출하시오.
> - 고객 ID(customer_id)가 "CS018205000001".

---
> R-005: 영수증 내역 데이터(df_receipt)에서 판매일(sales_ymd), 고객 ID(customer_id), 상품 코드(product_cd), 판매 금액(amount)의 순서로 열을 지정하고, 아래 조건을 모두 만족하는 데이터를 추출하라.
> - 고객 ID(customer_id)가 "CS018205000001"
> - 매출 금액(amount)이 1,000 이상

---
> R-006: 영수증 내역 데이터(df_receipt)에서 판매일(sales_ymd), 고객 ID(customer_id), 상품 코드(product_cd), 판매 수량(quantity), 판매 금액(amount) 순으로 열을 지정하고, 다음 조건을 모두 만족하는 데이터를 추출하라. 추출하라.
> - 고객 ID(customer_id)가 "CS018205000001".
> - 판매금액(amount)이 1,000 이상 또는 판매수량(quantity)이 5 이상

---
> R-007: 영수증 내역 데이터(df_receipt)에서 판매일(sales_ymd), 고객 ID(customer_id), 상품 코드(product_cd), 판매 금액(amount)의 순서로 열을 지정하고, 아래 조건을 모두 만족하는 데이터를 추출하라.
> - 고객 ID(customer_id)가 "CS018205000001"
> -  매출 금액(amount)이 1,000 이상 2,000 이하

---
> R-008: 영수증 내역 데이터(df_receipt)에서 판매일(sales_ymd), 고객 ID(customer_id), 상품 코드(product_cd), 판매 금액(amount)의 순서로 열을 지정하고, 아래 조건을 모두 만족하는 데이터를 추출하라.
> - 고객 ID(customer_id)가 "CS018205000001"
> - 상품코드(product_cd)가 "P071401019" 이외

---
> R-009: 다음 처리에서 출력 결과를 바꾸지 않고 OR을 AND로 다시 작성하시오.

`df_store %>% filter(!(prefecture_cd == "13" | floor_area > 900))`

---
> R-010: 매장 데이터(df_store)에서 매장 코드(store_cd)가 "S14"로 시작하는 항목만 전체 추출하여 10개를 표시하라.

---
> R-011: 고객 데이터(df_customer)에서 고객 ID(customer_id)의 끝자리가 1인 항목만 전체 추출하여 10건을 표시하라.

---
> R-012: 점포 데이터(df_store)에서 주소(address)에 '横浜市'가 포함된 항목만 모두 표시한다.

---
> R-013: 고객 데이터(df_customer)에서 상태코드(status_cd)가 알파벳 A~F로 시작하는 데이터를 모두 추출하여 10건을 표시하라.

---
> R-014: 고객 데이터(df_customer)에서 상태코드(status_cd)의 끝자리가 숫자 1~9로 끝나는 데이터를 모두 추출하여 10건을 표시하라.

---
> R-015: 고객 데이터(df_customer)에서 상태코드(status_cd)의 시작이 알파벳 A에서 F로 시작하고 끝이 숫자 1~9로 끝나는 데이터를 모두 추출하여 10건을 표시하라.

---
> R-016: 매장 데이터(df_store)에서 전화번호(tel_no)가 3자리-3자리-4자리인 데이터를 모두 표시하라.

---
> R-017: 고객 데이터(df_customer)를 생년월일(birth_day)을 기준으로 나이순으로 정렬하고, 맨 위부터 10개의 항목을 모두 표시하라.

---
> R-018: 고객 데이터(df_customer)를 생년월일(birth_day)을 기준으로 젊은 순서로 정렬하고, 맨 위부터 10개의 항목을 모두 표시하라.

---
> R-019: 영수증 내역 데이터(df_receipt)에 대해 건당 매출 금액(amount)이 높은 순서대로 순위를 부여하고, 맨 위부터 10건씩 표시한다. 항목은 고객 ID(customer_id), 매출 금액(amount), 부여한 랭크를 표시하도록 한다. 단, 매출액(amount)이 동일한 경우 동일한 순위를 부여한다.

---
> R-020: 영수증 내역 데이터(df_receipt)에 대해 건당 매출 금액(amount)이 높은 순서대로 순위를 부여하고, 맨 위부터 10건씩 표시한다. 항목은 고객 ID(customer_id), 매출 금액(amount), 부여한 순위를 표시한다. 단, 매출금액(amount)이 동일한 경우에도 별도의 순위를 부여해야 한다.

---
> R-021: 영수증 내역 데이터(df_receipt)에 대해 건수를 세어라.

---
> R-022: 영수증 내역 데이터(df_receipt)의 고객 ID(customer_id)에 대해 고유 건수를 세어라.

---
> R-023: 영수증 내역 데이터(df_receipt)에 대해 매장 코드(store_cd)별 판매 금액(amount)과 판매 수량(quantity)을 합산하라.

---
> R-024: 영수증 내역 데이터(df_receipt)에 대해 고객 ID(customer_id)별로 가장 최근 매출 날짜(sales_ymd)를 구하여 10건 표시하라.

---
> R-025: 영수증 내역 데이터(df_receipt)에 대해 고객 ID(customer_id)별로 가장 오래된 매출 날짜(sales_ymd)를 구하여 10건 표시하라.

---
> R-026: 영수증 내역 데이터(df_receipt)에 대해 고객 ID(customer_id)별로 가장 최근 매출 날짜(sales_ymd)와 가장 오래된 매출 날짜(sales_ymd)를 구하고, 양자가 서로 다른 데이터 10건을 표시하라.

---
> R-027: 영수증 내역 데이터(df_receipt)에 대해 매장 코드(store_cd)별 매출 금액(amount)의 평균을 계산하여 내림차순으로 TOP5를 표시하라.

---
> R-028: 영수증 내역 데이터(df_receipt)에 대해 매장 코드(store_cd)별 매출 금액(amount)의 중앙값을 계산하여 내림차순으로 TOP5를 표시하라.

---
> R-029: 영수증 내역 데이터(df_receipt)에 대해 매장 코드(store_cd)별로 상품 코드(product_cd)의 최빈값을 구하여 10건씩 표시하라.

---
> R-030: 영수증 내역 데이터(df_receipt)에 대해 매장 코드(store_cd)별 매출 금액(amount)의 분산을 계산하여 내림차순으로 5개씩 표시하시오.

---
> R-031: 영수증 내역 데이터(df_receipt)에 대해 매장 코드(store_cd)별 매출 금액(amount)의 표준편차를 계산하여 내림차순으로 5건씩 표시하라.

---
> R-032: 영수증 내역 데이터(df_receipt)의 매출 금액(amount)에 대해 25% 단위로 백분위수 값을 구하라.

---
> R-033: 영수증 내역 데이터(df_receipt)에 대해 매장 코드(store_cd)별 매출 금액(amount)의 평균을 계산하여 330개 이상을 추출하라.

---
> R-034: 영수증 내역 데이터(df_receipt)에 대해 고객 ID(customer_id)별로 매출 금액(amount)을 합산하여 전체 고객의 평균을 구하시오. 단, 고객 ID가 "Z"로 시작하는 것은 비회원을 의미하므로 제외하여 계산한다.

---
> R-035: 영수증 내역 데이터(df_receipt)에 대해 고객 ID(customer_id)별로 매출 금액(amount)을 합산하여 전체 고객의 평균을 구하고, 평균 이상 쇼핑을 한 고객을 추출하여 10건씩 표시하시오. 단, 고객 ID가 "Z"로 시작하는 것은 비회원을 의미하므로 제외하여 계산한다.

---
> R-036: 영수증 내역 데이터(df_receipt)와 매장 데이터(df_store)를 내부적으로 결합하여 영수증 내역 데이터의 모든 항목과 매장 데이터의 매장명(store_name)을 10건씩 표시하라.

---
> R-037: 상품 데이터(df_product)와 카테고리 데이터(df_category)를 내부적으로 결합하여 상품 데이터의 전체 항목과 카테고리 데이터의 카테고리 소분류 이름(category_small_name) 10개를 표시하라.

---
> R-038: 고객 데이터(df_customer)와 영수증 내역 데이터(df_receipt)에서 고객별 매출 금액의 총합을 구하여 10건을 표시하시오. 단, 판매 실적이 없는 고객에 대해서는 판매금액을 0으로 표시한다. 또한, 고객은 성별 코드(gender_cd)가 여성(1)인 고객을 대상으로 하며, 비회원(고객 ID가 "Z"로 시작하는 고객)은 제외한다.

---
> R-039: 영수증 내역 데이터(df_receipt)에서 매출 일수가 많은 고객 상위 20건을 추출한 데이터와 매출 금액 합계가 많은 고객 상위 20건을 추출한 데이터를 각각 생성하고, 이 두 가지를 완전히 외부 결합하라. 단, 비회원(고객 ID가 "Z"로 시작하는 고객)은 제외한다.

---
> R-040: 모든 매장과 모든 상품을 조합한 데이터를 생성하고 싶다. 매장 데이터(df_store)와 상품 데이터(df_product)를 직적분하여 건수를 계산하라.

---
> R-041: 영수증 내역 데이터(df_receipt)의 매출 금액(amount)을 날짜(sales_ymd)별로 집계하여, 지난번 매출이 있었던 날로부터의 매출 금액 증감을 계산하라. 그리고 결과를 10건 표시하라.

---
> R-042: 영수증 내역 데이터(df_receipt)의 매출 금액(amount)을 날짜(sales_ymd)별로 집계하고, 각 날짜의 데이터에 대해 이전, 전전, 3회 전에 매출이 있었던 날의 데이터를 결합하라. 그리고 결과를 10건 표시하라.

---
> R-043: 영수증 내역 데이터(receipt)와 고객 데이터(customer)를 결합하여 성별 코드(gender_cd)와 연령(age에서 계산)별 매출 금액(amount)을 합산한 매출 요약 데이터를 생성한다. 성별 코드는 0은 남성, 1은 여성, 9는 알 수 없음을 나타낸다.
>
> 단, 항목 구성은 연령, 여성 매출금액, 남성 매출금액, 여성 매출금액, 성별 미상 매출금액의 4개 항목으로 구성한다(세로로 연령, 가로로 성별 교차집계). 또한 연령은 10세 단위의 계급으로 한다.

---
> R-044: 043에서 만든 매출 요약 데이터(df_sales_summary)는 성별 매출을 가로로 나열한 데이터였다. 이 데이터에서 성별을 세로로 가져와서 연령, 성별 코드, 매출 금액의 세 가지 항목으로 변환하라. 단, 성별 코드는 남성을 '00', 여성을 '01', 알 수 없음을 '99'로 한다.

---
> R-045: 고객 데이터(df_customer)의 생년월일(birth_day)은 날짜형으로 데이터를 보유하고 있다. 이를 YYYYMMDD 형식의 문자열로 변환하여 고객 ID(customer_id)와 함께 10건 표시하라.

---
> R-046: 고객 데이터(df_customer)의 신청일(application_date)은 YYYYMMDD 형식의 문자열 형태로 데이터를 보유하고 있다. 이를 날짜형으로 변환하여 고객 ID(customer_id)와 함께 10건씩 표시하라.

---
> R-047: 영수증 내역 데이터(df_receipt)의 매출일(sales_ymd)은 YYYYMMDD 형식의 숫자형으로 데이터를 보유하고 있다. 이를 날짜형으로 변환하여 영수증 번호(receipt_no), 영수증 하위번호(receipt_sub_no)와 함께 10건씩 표시하라.

---
> R-048: 영수증 내역 데이터(df_receipt)의 매출 에포크 초(sales_epoch)는 숫자형 UNIX 초로 데이터를 보유하고 있다. 이를 날짜형으로 변환하여 영수증 번호(receipt_no), 영수증 서브번호(receipt_sub_no)와 함께 10건을 표시하라.

---
> R-049: 영수증 내역 데이터(df_receipt)의 매출 에포크 초(sales_epoch)를 날짜형으로 변환하여 '연도'만 추출하여 영수증 번호(receipt_no), 영수증 하위 번호(receipt_sub_no)와 함께 10건 표시하라.

---
> R-050: 영수증 내역 데이터(df_receipt)의 매출 에포크 초(sales_epoch)를 날짜형으로 변환하여 '월'만 추출하여 영수증 번호(receipt_no), 영수증 하위 번호(receipt_sub_no)와 함께 10건을 표시한다. 단, '월'은 0으로 채워진 2자리로 추출한다.

---
> R-051: 영수증 내역 데이터(df_receipt)의 매출 에포크 초를 날짜형으로 변환하여 '일'만 추출하여 영수증 번호(receipt_no), 영수증 하위 번호(receipt_sub_no)와 함께 10건 표시한다. 단, '일'은 0으로 채워진 2자리로 추출한다.

---
> R-052: 영수증 내역 데이터(df_receipt)의 매출 금액(amount)을 고객 ID(customer_id)별로 합산한 후, 매출 금액 총합에 대해 2,000원 이하를 0, 2,000원보다 큰 금액을 1로 이분화하여 고객 ID, 매출 금액 총합과 함께 10건씩 표시한다. 단, 고객 ID가 "Z"로 시작하는 것은 비회원을 의미하므로 제외하여 계산한다.

---
> R-053: 고객 데이터(df_customer)의 우편번호(postal_cd)에 대해 도쿄(앞 3자리가 100~209인 것)를 1, 그 외의 것을 0으로 이진화하라. 또한 영수증 내역 데이터(df_receipt)와 결합하여 전체 기간 동안 매출 실적이 있는 고객 수를 생성한 이항대수별로 계산하라.

---
> R-054: 고객 데이터(df_customer)의 주소(address)는 사이타마현, 지바현, 도쿄도, 가나가와현 중 하나이다. 도도부현별로 코드 값을 생성하여 고객 ID, 주소와 함께 10건씩 표시하라. 값은 사이타마현을 11, 지바현을 12, 도쿄도를 13, 가나가와현을 14로 한다.

---
> R-055: 영수증 명세서(df_receipt) 데이터의 매출 금액(amount)을 고객 ID(customer_id)별로 합산하고, 그 합산 금액의 사분위수를 구하시오. 그 후, 고객별 매출금액 합계에 대해 아래 기준으로 카테고리 값을 생성하여 고객 ID, 매출금액 합계와 함께 10건씩 표시하라. 카테고리 값은 순서대로 1~4로 한다.
> 
> - 최소값 이상 1사분위수 미만 ・・・ 1을 부여
> - 1사분위 이상 2사분위 미만 ・・・ 2를 부여
> - 2사분위 이상 3사분위 미만 ・・・ 3을 부여
> - 3사분위 이상 ・・・ 4 부여

---
> R-056: 고객 데이터(df_customer)의 나이(age)를 기준으로 10세 단위로 연령을 계산하여 고객 ID(customer_id), 생년월일(birth_day)과 함께 10건씩 표시한다. 단, 60세 이상은 모두 60대 이상으로 한다. 연령을 나타내는 카테고리 명칭은 임의대로 한다.

---
> R-057: 056의 추출 결과와 성별 코드(gender_cd)에 따라 성별×연령의 조합을 나타내는 카테고리 데이터를 새로 만들어 10개를 표시하라. 조합을 나타내는 카테고리의 값은 임의로 정한다.

---
> R-058: 고객 데이터(df_customer)의 성별 코드(gender_cd)를 더미 변수로 만들어 고객 ID(customer_id)와 함께 10건 표시하라.

---
> R-059: 영수증 내역 데이터(df_receipt)의 매출 금액(amount)을 고객 ID(customer_id)별로 합산하고, 매출 금액 합계를 평균 0, 표준편차 1로 표준화하여 고객 ID, 매출 금액 합계와 함께 10건씩 표시하라. 표준화에 사용하는 표준편차는 분산 제곱근 또는 불균형 분산 제곱근 중 어느 것이든 상관없다. 단, 고객 ID가 "Z"로 시작하는 것은 비회원을 의미하므로 제외하여 계산한다.

---
> R-060: 영수증 내역 데이터(df_receipt)의 매출 금액(amount)을 고객 ID(customer_id)별로 합산하여 매출 금액 합계를 최소값 0, 최대값 1로 정규화하여 고객 ID, 매출 금액 합계와 함께 10건씩 표시한다. 단, 고객 ID가 "Z"로 시작하는 것은 비회원을 의미하므로 제외하여 계산한다.

---
> R-061: 영수증 내역 데이터(df_receipt)의 매출 금액(amount)을 고객 ID(customer_id)별로 합산하고, 매출 금액 합계를 상수 대수화(하단 10)하여 고객 ID, 매출 금액 합계와 함께 10건씩 표시한다. 단, 고객 ID가 "Z"로 시작하는 것은 비회원을 의미하므로 제외하여 계산한다.

---
> R-062: 영수증 내역 데이터(df_receipt)의 매출금액(amount)을 고객ID(customer_id)별로 합산하고, 매출금액 합계를 자연대수화(하단 e)하여 고객ID, 매출금액 합계와 함께 10건씩 표시한다. 단, 고객 ID가 "Z"로 시작하는 것은 비회원을 의미하므로 제외하여 계산한다.

---
> R-063: 상품 데이터(df_product)의 단가(unit_price)와 원가(unit_cost)로부터 각 상품의 이익액을 산출하고, 그 결과를 10건 표시하시오.

---
> R-064: 상품 데이터(df_product)의 단가(unit_price)와 원가(unit_cost)에서 각 상품의 전체 평균 수익률을 계산하시오. 단, 단가와 원가에는 결손이 발생한다는 점에 유의하라.

---
> R-065: 상품 데이터(df_product)의 각 상품에 대해 수익률이 30%가 되는 새로운 단가를 구하시오. 단, 1원 미만은 반올림한다. 그리고 10개의 결과를 표시하고, 수익률이 대략 30% 정도인 것을 확인하라. 단, 단가(unit_price)와 원가(unit_cost)에는 적자가 발생한다는 점에 유의하라.

---
> R-066: 상품 데이터(df_product)의 각 상품에 대해 수익률이 30%가 되는 새로운 단가를 구하시오. 이번에는 1원 미만은 반올림한다(반올림 또는 짝수로 반올림해도 무방하다). 그리고 10개의 결과를 표시하게 하고, 수익률이 대략 30% 정도인 것을 확인하라. 단, 단가(unit_price)와 원가(unit_cost)에는 결손이 발생한다는 점에 유의한다.

---
> R-067: 상품 데이터(df_product)의 각 상품에 대해 수익률이 30%가 되는 새로운 단가를 구하시오. 이번에는 1원 미만은 반올림한다. 그리고 10개의 결과를 표시하고, 수익률이 대략 30% 정도인 것을 확인하라. 단, 단가(unit_price)와 원가(unit_cost)에는 적자가 발생하고 있다는 점에 유의하라.

---
> R-068: 상품 데이터(df_product)의 각 상품에 대해 소비세율 10%의 부가세 포함 금액을 구하고, 1원 미만의 단수는 절사하여 10개의 결과를 표시하시오. 단, 단가(unit_price)에는 결손이 발생한다는 점에 유의하라.

---
> R-069: 영수증 내역 데이터(df_receipt)와 상품 데이터(df_product)를 결합하여 고객별 전체 상품 판매금액 합계와 카테고리 대분류 코드(category_major_cd)가 "07"(병조림 통조림)인 상품의 판매금액 합계를 계산한 후, 양자의 비율을 구하시오. 추출 대상은 카테고리 대분류 코드 "07"(병조림 통조림)의 판매 실적이 있는 고객으로 한정하고, 결과를 10건만 표시한다.

---
> R-070: 영수증 내역 데이터(df_receipt)의 매출일(sales_ymd)에 대해 고객 데이터(df_customer)의 회원가입일(application_date)로부터의 경과일수를 계산하여 고객 ID(customer_id), 매출일, 회원가입일과 함께 10건을 표시하라(sales_ymd는 수치, application_date는 문자열로 데이터를 보관하고 있다는 점에 유의).

---
> R-071: 영수증 내역 데이터(df_receipt)의 매출일(sales_ymd)에 대해 고객 데이터(df_customer)의 회원가입일(application_date)로부터의 경과 개월 수를 계산하여 고객 ID(customer_id), 매출일, 회원가입일과 함께 10건 표시 (sales_ymd는 숫자, application_date는 문자열로 데이터를 보관하는 점에 유의) 1개월 미만은 반올림한다.

---
> R-072: 영수증 내역 데이터(df_receipt)의 매출일(df_customer)에 대해 고객 데이터(df_customer)의 회원가입 신청일(application_date)로부터의 경과년수를 계산하여 고객 ID(customer_id), 매출일, 회원가입 신청일과 함께 10건 (sales_ymd는 수치, application_date는 문자열로 데이터를 보관하고 있다는 점에 유의) 1년 미만은 반올림한다.

---
> R-073: 영수증 내역 데이터(df_receipt)의 매출일(sales_ymd)에 대해 고객 데이터(df_customer)의 회원가입일(application_date)로부터의 에포크 초 단위의 경과 시간을 계산하여 고객 ID(customer_id), 매출일, 회원가입일과 함께 10건을 표시한다. 과 함께 10건을 표시하라(단, sales_ymd는 수치, application_date는 문자열로 데이터를 보유하고 있다는 점에 유의). 단, 시간 정보는 보유하지 않으므로 각 날짜는 0시 0분 0초로 표시한다.

---
> R-074: 영수증 내역 데이터(df_receipt)의 매출일(sales_ymd)에 대해 해당 주 월요일부터의 경과일수를 계산하여 매출일, 직전 월요일까지 10건씩 표시하라(sales_ymd는 수치로 데이터를 보관하고 있다는 점에 유의).

---
> R-075: 고객 데이터(df_customer)에서 무작위로 1%의 데이터를 추출하여 맨 앞부터 10개를 표시하라.

---
> R-076: 고객 데이터(df_customer)에서 성별 코드(gender_cd)의 비율에 따라 무작위로 10%의 데이터를 층화 추출하고, 성별 코드별로 건수를 집계하라.

---
> R-077: 영수증 명세서 데이터(df_receipt)의 매출 금액을 고객 단위로 합산하고, 합산한 매출 금액의 편차를 추출하라. 단, 이상값은 매출금액 합계를 로그화한 후 평균과 표준편차를 계산하여 그 평균에서 3σ 이상 벗어난 것으로 한다(자연대수, 상용대수 모두 가능). 결과는 10건 표시하라.

---
> R-078: 영수증 내역 데이터(df_receipt)의 매출 금액(amount)을 고객 단위로 합산하고, 합산된 매출 금액의 외곽값을 추출한다. 단, 고객 ID가 "Z"로 시작하는 것은 비회원을 의미하므로 제외하여 계산한다. 여기서 이상값은 1사분위와 3사분위의 차이인 IQR을 이용하여 '1사분위수 -1.5×IQR' 이하 또는 '3사분위수+1.5×IQR'을 초과하는 것으로 한다. 결과는 10건 표시한다.

---
> R-079: 상품 데이터(df_product)의 각 항목에 대해 결손 수를 확인하라.

---
> R-080: 상품 데이터(df_product) 중 어느 한 항목에 결손이 발생한 레코드를 모두 삭제한 새로운 상품 데이터를 생성한다. 또한, 삭제 전후의 건수를 표시하고, 079에서 확인한 건수만큼 감소한 것도 확인해야 한다.

---
> R-081: 단가(unit_price)와 원가(unit_cost)의 결손값에 대해 각각의 평균값으로 보완한 새로운 상품 데이터를 작성하시오. 단, 평균값은 1원 미만은 반올림한다(반올림 또는 짝수로 반올림해도 무방하다). 보완 실시 후 각 항목에 대해 결손이 발생하지 않았는지도 확인해야 한다.

---
> R-082: 단가(unit_price)와 원가(unit_cost)의 결손값에 대해 각각의 중앙값으로 보완한 새로운 상품 데이터를 작성하시오. 단, 중앙값은 1원 미만은 반올림한다(반올림 또는 짝수로 반올림해도 무방하다). 보완 실시 후 각 항목에 대해 결손이 발생하지 않았는지도 확인해야 한다.

---
> R-083: 단가(unit_price)와 원가(unit_cost)의 결손값에 대해 각 상품의 카테고리 소분류 코드(category_small_cd)별로 산출한 중간값으로 보완한 새로운 상품 데이터를 작성한다. 단, 중앙값은 1원 미만은 반올림한다(반올림 또는 짝수로 반올림해도 무방하다). 보완 실시 후 각 항목에 대해 결손이 발생하지 않았는지도 확인해야 한다.

---
> R-084: 고객 데이터(df_customer)의 전체 고객에 대해 전체 기간의 매출 금액에서 2019년 매출 금액이 차지하는 비율을 계산하여 새로운 데이터를 생성한다. 단, 매출 실적이 없는 경우 0으로 처리한다. 그리고 계산한 비율이 0을 초과하는 것을 추출하여 결과를 10건씩 표시하라. 또한, 작성된 데이터에 결손이 없는지 확인하라.

---
> R-085: 고객 데이터(df_customer)의 모든 고객에 대해 우편번호(postal_cd)를 이용하여 지오코드 데이터(df_geocode)를 연결하여 새로운 고객 데이터를 생성한다. 단, 하나의 우편번호(postal_cd)에 여러 개의 경도(longitude), 위도(latitude) 정보가 연결된 경우에는 경도(longitude), 위도(latitude)의 평균값을 산출하여 사용해야 한다. 또한, 생성된 결과를 확인하기 위해 10개의 결과를 표시한다.

---
> R-086: 085에서 생성한 위도경도별 고객 데이터에 대해 회원 신청 매장 코드(application_store_cd)를 키로 매장 데이터(df_store)와 결합하라. 그리고 신청 매장의 위도(latitude)-경도 정보(longitude)와 고객 주소(address)의 위도-경도를 이용하여 신청 매장과 고객 주소의 거리(단위: km)를 구하고, 고객 ID(customer_id), 고객 주소(address), 매장 주소(address)와 함께 표시하라. 과 함께 표시하라. 계산식은 아래의 간단한 식을 사용하되, 그 외 정밀도가 높은 방식을 이용한 라이브러리를 사용해도 무방하다. 결과는 10건을 표시한다.

$$
\begin{aligned}
& \mbox{위도(라디안)} : \phi \\ \\
& \mbox{경도(라디안)}：\lambda \\
& \mbox{거리}L = 6371 * \arccos(\sin \phi_1 * \sin \phi_2 \\)
& \cos \phi_1 * \cos \phi_2 * \cos(\lambda_1 - \lambda_2))
\end{aligned}
$$

---
> R-087: 고객 데이터(df_customer)에는 다른 매장에서의 신청 등으로 동일 고객이 여러 개 등록되어 있다. 이름(customer_name)과 우편번호(postal_cd)가 같은 고객은 동일 고객으로 간주하여 1고객 1레코드가 되도록 이름을 붙인 명목 고객 데이터를 생성하고, 고객 데이터 개수, 명목 고객 데이터 개수, 중복 횟수를 계산하라. 단, 동일 고객에 대해서는 매출 금액 합계가 가장 높은 것을 남기고, 매출 금액 합계가 동일하거나 매출 실적이 없는 고객에 대해서는 고객 ID(customer_id)의 번호가 작은 것을 남긴다.

---
> R-088: 087에서 생성한 데이터를 바탕으로 고객 데이터에 통합명칭 ID를 부여한 데이터를 생성한다. 단, 통합명칭 ID는 아래의 사양으로 부여한다.
>  중복되지 않은 고객
> - 중복되지 않은 고객 : 고객 ID(customer_id) 설정
> - 중복되는 고객: 이전 설문에서 추출한 레코드의 고객 ID를 설정한다.
>고객 ID의 고유 개수 
> 고객 ID의 고유 건수와 통합명칭 ID의 고유 건수 차이도 확인해야 한다.

---
> R-089: 예측 모델 구축을 위해 판매 실적이 있는 고객을 학습용 데이터와 테스트용 데이터로 나누고 싶다. 각각 8:2의 비율로 무작위로 데이터를 분할하라.

---
> R-090: 영수증 내역 데이터(df_receipt)는 2017년 1월 1일부터 2019년 10월 31일까지의 데이터를 가지고 있다. 매출 금액(amount)을 월별로 집계하여 학습용 12개월, 테스트용 6개월의 시계열 모델 구축용 데이터 3세트를 생성한다.

---
> R-091: 고객 데이터(df_customer)의 각 고객에 대해 매출 실적이 있는 고객 수와 매출 실적이 없는 고객 수가 1:1이 되도록 언더샘플링으로 추출하라.

---
> R-092: 고객 데이터(df_customer)의 성별을 제3정규형으로 정규화하라.

---
> R-093: 상품 데이터(df_product)는 각 카테고리의 코드 값만 보유하고 카테고리 이름은 보유하지 않는다. 카테고리 데이터(df_category)와 결합하여 비정규화하여 카테고리 이름을 보유한 새로운 상품 데이터를 생성한다.

---
> R-094: 093에서 생성된 카테고리 이름을 가진 제품 데이터를 다음 사양으로 파일로 정리하라.
>
> |파일 형식|헤더 포함/미포함|문자 인코딩|
> |:--:|:--:|:--:|
> |CSV(쉼표로 구분된 값)|있음|UTF-8|
>
> 파일 출력 대상의 경로는 다음과 같아야 합니다.
> 
> |출력 대상|
> |:--:|
> |. /data|

---
> R-095: 093에서 생성된 카테고리 이름을 가진 제품 데이터를 다음 사양으로 파일로 정리합니다.
>
> |파일 형식|헤더 포함/미포함|문자 인코딩|
> |:--:|:--:|:--:|
> |CSV(쉼표로 구분된 값)|있음|CP932|
>
> 파일 출력 대상의 경로는 다음과 같아야 합니다.
> 
> |출력 대상|
> |:--:|
> |. /data|

---
> R-096: 093에서 생성된 카테고리 이름을 가진 제품 데이터를 다음 사양으로 파일로 정리하라.

> |파일 형식|헤더 포함/미포함|문자 인코딩|
> |:--:|:--:|:--:|
> |CSV(쉼표로 구분된 값)|없음|UTF-8|
>
> 파일 출력 대상의 경로는 다음과 같아야 합니다.
> 
> |출력 대상|
> |:--:|
> |. /data|

---
> R-097: 094에서 만든 다음 형식의 파일을 로드하고 세 개의 데이터 항목을 표시한 다음 올바르게 가져왔는지 확인합니다.
> 
> |파일 형식|헤더 포함/미포함|문자 인코딩|
> |:--:|:--:|:--:|
> |CSV(쉼표로 구분된 값)|있음|UTF-8|

---
> R-098: 096에서 만든 다음 형식의 파일을 로드하고 세 개의 데이터 항목을 표시한 다음 올바르게 가져왔는지 확인합니다.
> 
> |파일 형식|헤더 포함/미포함|문자 인코딩|
> |:--:|:--:|:--:|
> |CSV(쉼표로 구분된 값)|있음|UTF-8|

---
> R-099: 093에서 생성된 카테고리 이름을 가진 제품 데이터를 다음 사양으로 파일로 정리합니다.

> |파일 형식|헤더 포함/미포함|문자 인코딩|
> |:--:|:--:|:--:|
> |TSV(탭으로 구분된 값)|있음|UTF-8|
>
> 파일 출력 대상의 경로는 다음과 같아야 합니다.
> 
> |출력 대상|
> |:--:|
> |. /data|


---
> R-100: 093에서 생성된 카테고리 이름을 가진 제품 데이터를 다음 사양의 파일로 정리합니다.
> 
> |파일 형식|헤더 포함/미포함|문자 인코딩|
> |:--:|:--:|:--:|
> |TSV(탭으로 구분된 값)|있음|UTF-8|

# 이것으로 100번 노크는 끝났습니다. 수고하셨습니다!